In [1]:
# copyrights (c) 2023 - thyung
# GPLv3

from ipywidgets import *
import time
import threading
import math
import random

SCREEN_X = 24
SCREEN_Y = 4
FRAME_RATE = 10
FIRE_SPEED_X = 2.0 / FRAME_RATE
PARTICLE_SPEED_X = 0.5 / FRAME_RATE
MIN_PARTICLE_DISTANCE = 10
MAX_MISSED = 3

g_lbl_inst0 = Label(value='Summary: space ship fires "gate" to turn particles "state" to |0>')
g_lbl_inst1 = Label(value='1. Move ship "D" to collect particle.')
g_lbl_inst2 = Label(value='2. When ship hits particle, it measures the state and get +1 score for |0> and -1 for |1>.')
g_lbl_inst3 = Label(value='3. Fire gate to turn particle state to |0> and collect it.')
g_lbl_inst4 = Label(value='4. If miss 3 particles, game over.')

g_states = ['0', '1', '+', '-', 'R', 'L']

g_exit = False
g_fires = []
g_particles = []
g_totalscore = 0
g_highestscore = 0
g_missed = 0
g_status = ''
g_tiles = [[Button(description = ' ', layout=Layout(width='30px', height='30px'), disabled=True) for i in range(SCREEN_X)] 
            for j in range(SCREEN_Y)]
boxes = [Box(i) for i in g_tiles]
board = VBox(boxes)

box_inst = VBox([g_lbl_inst0, g_lbl_inst1, g_lbl_inst2, g_lbl_inst3, g_lbl_inst4])
box_board_inst = VBox([board, box_inst])
g_btn_newgame = Button(description='new game')
g_btn_up = Button(description='^', layout=Layout(width='40px'))
g_btn_down = Button(description='v', layout=Layout(width='40px'))
g_btn_left = Button(description='<', layout=Layout(width='40px'))
g_btn_right = Button(description='>', layout=Layout(width='40px'))
panel_updown = VBox([g_btn_up, g_btn_down])
panel_dir = Box([g_btn_left, panel_updown, g_btn_right], layout=Layout(align_items='center'))
g_btn_x = Button(description='X')
g_btn_y = Button(description='Y')
g_btn_z = Button(description='Z')
g_btn_s = Button(description='S')
g_btn_h = Button(description='H')
g_btn_exit = Button(description='Exit')
g_btn_particle = Button(description='Particle')
g_txt_totalscore = BoundedIntText(value=0, description='score:', layout=Layout(width='140px'), disabled=True)
g_txt_highestscore = BoundedIntText(value=0, description='highest:', layout=Layout(width='140px'), disabled=True)
g_txt_missed = BoundedIntText(value=0, description='missed:', layout=Layout(width='140px'), disabled=True)
g_lbl_status = Label(value=g_status)
ctrl_panel = VBox([g_btn_newgame, 
                    panel_dir, 
                    g_btn_x, g_btn_y, g_btn_z, g_btn_s, g_btn_h, 
                    # g_btn_exit, 
                    # g_btn_particle, 
                    g_txt_totalscore, g_txt_highestscore, 
                    g_txt_missed, g_lbl_status],
                    layout=Layout(align_items='center'))
screen = Box([ctrl_panel, box_board_inst], layout=Layout(justify_content='space-between', align_items='center'))
display(screen)
s_states = ['0', '1', '+', '-', 'R', 'L']

class MovingObject:
    def __init__(self, x, y, speed_x, speed_y):
        self.x = x
        self.y = y
        self.speed_x = speed_x
        self.speed_y = speed_y
    def nextframe(self):
        self.x = self.x + self.speed_x
        self.y = self.y + self.speed_y
    def is_inside_screen(self):
        if 0 <= self.x and self.x < SCREEN_X and 0 <= self.y and self.y < SCREEN_Y:
            return True
        else:
            return False

class Particle(MovingObject):
    def __init__(self, x, y, speed_x, speed_y, state):
        super().__init__(x, y, speed_x, speed_y)
        self.state = state
    def __str__(self):
        return '(x,y)=({},{}), (sx,sy)=({},{}), state={}'.format(self.x, self.y, self.speed_x, self.speed_y, self.state)

class SpaceShip:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def move_up(self):
        self.y = self.y - 1
        if self.y < 0:
            self.y = self.y + SCREEN_Y
    def move_down(self):
        self.y = self.y + 1
        if SCREEN_Y <= self.y:
            self.y = self.y - SCREEN_Y
    def move_left(self):
        self.x = self.x - 1
        if self.x < 0:
            self.x = self.x + SCREEN_X
    def move_right(self):
        self.x = self.x + 1
        if SCREEN_X <= self.x:
            self.x = self.x - SCREEN_X
    def draw(self):
        draw_screen(self.x, self.y, 'D')

class Fire(MovingObject):
    def __init__(self, x, y, speed_x, speed_y, gate):
        super().__init__(x, y, speed_x, speed_y)
        self.gate = gate
    def __str__(self):
        return '(x,y)=({},{}), (sx,sy)=({},{}), gate={}'.format(self.x, self.y, self.speed_x, self.speed_y, self.gate)

def draw_screen(x, y, v):
    global g_tiles
    g_tiles[math.floor(y)][math.floor(x)].description = v

def init_screen():
    for y in range(SCREEN_Y):
        for x in range(SCREEN_X):
            draw_screen(x, y, ' ')

def on_click_newgame(b):
    global g_fires
    global g_particles
    global g_totalscore
    global g_txt_totalscore
    global g_missed
    global g_lbl_status
    global g_exit
    g_fires = []
    g_particles = []
    g_totalscore = 0
    g_txt_totalscore.value = g_totalscore
    g_missed = 0
    g_txt_missed.value = 0
    g_lbl_status.value = ''
    g_exit = False
    threading.Thread(target=game_loop).start()

def on_click_up(b):
    global g_ship
    g_ship.move_up()

def on_click_down(b):
    global g_ship
    g_ship.move_down()

def on_click_left(b):
    global g_ship
    g_ship.move_left()

def on_click_right(b):
    global g_ship
    g_ship.move_right()

def on_click_x(b):
    global g_fires
    fire = Fire(g_ship.x + 1, g_ship.y, FIRE_SPEED_X, 0, 'X')
    g_fires.append(fire)

def on_click_y(b):
    global g_fires
    fire = Fire(g_ship.x + 1, g_ship.y, FIRE_SPEED_X, 0, 'Y')
    g_fires.append(fire)

def on_click_z(b):
    global g_fires
    fire = Fire(g_ship.x + 1, g_ship.y, FIRE_SPEED_X, 0, 'Z')
    g_fires.append(fire)

def on_click_s(b):
    global g_fires
    fire = Fire(g_ship.x + 1, g_ship.y, FIRE_SPEED_X, 0, 'S')
    g_fires.append(fire)

def on_click_h(b):
    global g_fires
    fire = Fire(g_ship.x + 1, g_ship.y, FIRE_SPEED_X, 0, 'H')
    g_fires.append(fire)

def on_click_exit(b):
    global g_exit
    g_exit = True

def on_click_particle(b):
    global g_particles
    particle = Particle(SCREEN_X - 1, g_ship.y, -PARTICLE_SPEED_X, 0, '0' )
    g_particles.append(particle)

def nextframe_fires():
    global g_fires
    for fire in g_fires:
        fire.nextframe()
    g_fires = [fire for fire in g_fires if fire.is_inside_screen()]
def nextframe_particles():
    global g_particles
    global g_missed
    global g_exit
    for particle in g_particles:
        particle.nextframe()
    particles_missed = [p for p in g_particles if not p.is_inside_screen()]
    missed = len(particles_missed)
    if 0 < missed:
        g_missed = g_missed + missed
        g_txt_missed.value = g_missed
    if MAX_MISSED <= g_missed:
        g_lbl_status.value = 'GAME OVER'
        g_exit = True
    g_particles = [particle for particle in g_particles if particle.is_inside_screen()]

def draw_frame():
    global g_fires
    global g_particles
    for fire in g_fires:
        draw_screen(fire.x, fire.y, fire.gate)
    for particle in g_particles:
        draw_screen(particle.x, particle.y, particle.state)

def apply_gate_on_state(gate, state):
    if gate == 'X':
        if state == '0':
            result = '1'
        elif state == '1':
            result = '0'
        elif state == '+':
            result = '+'
        elif state == '-':
            result = '-'
        elif state == 'R':
            result = 'L'
        elif state == 'L':
            result = 'R'
    elif gate == 'Y':
        if state == '0':
            result = '1'
        elif state == '1':
            result = '0'
        elif state == '+':
            result = '-'
        elif state == '-':
            result = '+'
        elif state == 'R':
            result = 'R'
        elif state == 'L':
            result = 'L'
    elif gate == 'Z':
        if state == '0':
            result = '0'
        elif state == '1':
            result = '1'
        elif state == '+':
            result = '-'
        elif state == '-':
            result = '+'
        elif state == 'R':
            result = 'L'
        elif state == 'L':
            result = 'R'
    elif gate == 'S':
        if state == '0':
            result = '0'
        elif state == '1':
            result = '1'
        elif state == '+':
            result = 'R'
        elif state == '-':
            result = 'L'
        elif state == 'R':
            result = '-'
        elif state == 'L':
            result = '+'
    elif gate == 'H':
        if state == '0':
            result = '+'
        elif state == '1':
            result = '-'
        elif state == '+':
            result = '0'
        elif state == '-':
            result = '1'
        elif state == 'R':
            result = 'L'
        elif state == 'L':
            result = 'R'

    return result
        
def get_particle_score(state):
    if state == '0':
        score = 1
    elif state == '1':
        score = -1
    elif state == '+' or state == '-' or state == 'R' or state == 'L':
        score = random.randrange(-1, 2, 2)  # -1 or +1
    return score

def check_fire_collision():
    global g_fires
    global g_particles
    for fire in g_fires:
        fire_x = math.floor(fire.x)
        fire_y = math.floor(fire.y)
        for particle in g_particles:
            if math.floor(particle.x) == fire_x and math.floor(particle.y) == fire_y:
                result = apply_gate_on_state(fire.gate, particle.state)
                particle.state = result
                g_fires.remove(fire)
                break

def check_ship_collision():
    global g_particles
    global g_totalscore
    global g_highestscore
    ship_x = math.floor(g_ship.x)
    ship_y = math.floor(g_ship.y)
    for particle in g_particles:
        if math.floor(particle.x) == ship_x and math.floor(particle.y) == ship_y:
            score = get_particle_score(particle.state)
            g_totalscore = g_totalscore + score
            if g_totalscore < 0:
                g_totalscore = 0
            if g_highestscore < g_totalscore:
                g_highestscore = g_totalscore
                g_txt_highestscore.value = g_highestscore
            g_txt_totalscore.value = g_totalscore
            g_particles.remove(particle)
            break
    
def gen_particle():
    global g_totalscore
    global g_states
    global g_particles
    r = random.randrange(0, 100)
    if r <= g_totalscore or g_particles == []:
        x = SCREEN_X - 1
        y = random.randrange(0, SCREEN_Y)
        if [p for p in g_particles if SCREEN_X - MIN_PARTICLE_DISTANCE <= p.x and p.y == y] == []:
            particle = Particle(x, y, -PARTICLE_SPEED_X, 0, random.choice(g_states))
            g_particles.append(particle)

def game_loop():
    global g_exit
    while not g_exit:
        init_screen()
        nextframe_fires()
        check_fire_collision()
        nextframe_particles()
        check_fire_collision()
        gen_particle()
        check_ship_collision()
        draw_frame()
        g_ship.draw()
        time.sleep(1.0 / FRAME_RATE)
        
g_btn_newgame.on_click(on_click_newgame)
g_btn_up.on_click(on_click_up)
g_btn_down.on_click(on_click_down)
g_btn_left.on_click(on_click_left)
g_btn_right.on_click(on_click_right)

g_btn_exit.on_click(on_click_exit)

g_btn_x.on_click(on_click_x)
g_btn_y.on_click(on_click_y)
g_btn_z.on_click(on_click_z)
g_btn_s.on_click(on_click_s)
g_btn_h.on_click(on_click_h)

g_btn_particle.on_click(on_click_particle)


g_ship = SpaceShip(0, 0)
threading.Thread(target=game_loop).start()


Box(children=(VBox(children=(Button(description='new game', style=ButtonStyle()), Box(children=(Button(descrip…